### [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist)



https://github.com/zalandoresearch/fashion-mnist



In [2]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn, rnn
from tqdm import tqdm
from mxnet import  autograd

In [3]:
def transform_f_mnist(data, label):
    # height x weight x channel to channel x height x weight
    return(mx.nd.transpose(data.astype('float32'), (2,0,1))/255, label.astype('float32'))

In [4]:
test_set = gluon.data.vision.FashionMNIST(train=False, transform=transform_f_mnist)
train_set = gluon.data.vision.FashionMNIST(train=True, transform=transform_f_mnist)

/opt/venv/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
/opt/venv/lib/python3.6/site-packages/mxnet/gluon/data/vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


In [5]:
test_set_iter  = gluon.data.DataLoader(test_set,  batch_size=120, shuffle=True)
train_set_iter = gluon.data.DataLoader(train_set, batch_size=120, shuffle=True)

In [6]:
for i, (data_x, data_y) in enumerate(train_set_iter):
    data_set_x = data_x
    data_set_y = data_y
    break

In [7]:
data_set_x.shape

(120, 1, 28, 28)

In [8]:
class f_mnist(gluon.Block):
    def __init__(self, **kwargs):
        super(f_mnist, self).__init__(**kwargs)
        with self.name_scope():
            self.conv0 = nn.Conv2D(channels=64, kernel_size=5, activation='relu')
            self.maxpool = nn.MaxPool2D()
            self.conv1 = nn.Conv2D(channels=128, kernel_size=3, activation='relu')
            self.avgpool = nn.GlobalAvgPool2D()
            self.dense = nn.Dense(10)
            
    def forward(self, inputs):
        x = self.conv0(inputs)
        x = self.maxpool(x)
        x = self.conv1(x)
        x = self.avgpool(x)
        return self.dense(x)


In [9]:
ctx = mx.gpu(0)

In [10]:
net = f_mnist()

In [11]:
def evaluate(net, data_iter, ctx):
    acc = mx.metric.Accuracy()
    for data_x, data_y in data_iter:
        data_x = data_x.as_in_context(ctx)
        data_y = data_y.as_in_context(ctx)
        with autograd.predict_mode():
            output = net(data_x)
        predictions = mx.nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=data_y)
        mx.ndarray.waitall()
    return acc.get()[1]

In [12]:
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
trainer = gluon.trainer.Trainer(net.collect_params(), 'adam')
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
net(data_set_x.as_in_context(ctx)).shape, print(net)

f_mnist(
  (conv0): Conv2D(1 -> 64, kernel_size=(5, 5), stride=(1, 1))
  (maxpool): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False)
  (conv1): Conv2D(64 -> 128, kernel_size=(3, 3), stride=(1, 1))
  (avgpool): GlobalAvgPool2D(size=(1, 1), stride=(1, 1), padding=(0, 0), ceil_mode=True)
  (dense): Dense(128 -> 10, linear)
)


((120, 10), None)

In [ ]:
epoches = 20
for e in range(epoches):
    train_loss = 0
    for i, (data_x, data_y) in enumerate(tqdm(train_set_iter)):
        data = data_x.as_in_context(ctx)
        label = data_y.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss_ = loss(output, label)
        loss_.backward()
        trainer.step(data.shape[0])
        train_loss += mx.nd.mean(loss_).asscalar()
    train_acc = evaluate(net, test_set_iter,  ctx=ctx)
    test_acc  = evaluate(net, train_set_iter, ctx=ctx)
    
    print("{} : the train loss ; {}, the train accracy ; {}, the test accuracy ; {}".format(e, train_loss / len(train_set_iter), train_acc, test_acc))

  1%|          | 5/500 [00:00<00:09, 49.90it/s]

0 : the train loss ; 1.0270237883329392, the train accracy ; 0.7257, the test accuracy ; 0.7280833333333333


  1%|          | 5/500 [00:00<00:10, 49.43it/s]

1 : the train loss ; 0.7183285561203957, the train accracy ; 0.7554, the test accuracy ; 0.7484


  1%|          | 5/500 [00:00<00:10, 45.40it/s]

...

  1%|          | 5/500 [00:00<00:10, 49.20it/s]

15 : the train loss ; 0.4067530354857445, the train accracy ; 0.8623, the test accuracy ; 0.8592166666666666


  1%|          | 5/500 [00:00<00:09, 49.82it/s]

16 : the train loss ; 0.3995973082482815, the train accracy ; 0.8627, the test accuracy ; 0.85565


100%|██████████| 500/500 [00:07<00:00, 67.86it/s]
